In [9]:
import os
import sys
import json
from datetime import datetime
import subprocess
import pandas as pd
import plotly.graph_objects as go
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..', '..'))
sys.path.append(root_dir)
import Birdeye.Basics.ATH_Breakout_Master as ATH_Breakout_Master
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [10]:
# Specify the path to the Python file you want to run
python_file_path = 'Dataset_ATH_Breakout.py'

# Run the Python file
subprocess.run(['python', python_file_path])

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}

--------------------------------

Running on: solana

--------------------------------

NEW TOKEN FILTERS:

Min Liquidity: 30K
Max Liquidity: 1.00Mil
Min Market Cap: 1.00Mil
Max Market Cap: 10.00Mil
Filtering New Tokens for: 1 days, 18 hours, 0 minutes back
Token Launch Liquidity Filter: 30K

--------------------------------
 
 
--------------------------------



In [6]:
# Get the most recent folder
base_path = 'Data/New_Token_Data'
current_date = datetime.now().strftime('%Y_%m_%d')
date_folder = os.path.join(base_path, current_date)
ohlcv_folder = os.path.join(date_folder, 'OHLCV_Data')
ohlcv_datetime_folder = ATH_Breakout_Master.get_most_recent_folder(ohlcv_folder)

# Print the name of the folder being used for import
print(f"Importing OHLCV data from folder: {ohlcv_datetime_folder}")

# Import the OHLCV data
imported_ohlcv_data = ATH_Breakout_Master.import_ohlcv_data(ohlcv_datetime_folder)

Importing OHLCV data from folder: Data/New_Token_Data/2024_11_11/OHLCV_Data/2024_Nov_11_0503AM


In [ ]:
def plot_ath_breakouts(imported_ohlcv_data, initial_timeframe='5m'):
    """
    Plot price action showing previous ATH and new ATH from recent candles.
    Mobile-optimized with clean display and improved visuals.
    """
    
    def check_recent_ath(df, lookback=4):
        if len(df) <= lookback:
            return False, None, None
            
        # Get previous ATH (excluding last 4 candles)
        historical_df = df.iloc[:-lookback]
        previous_ath = historical_df['high'].max()
        previous_ath_idx = historical_df['high'].idxmax()
        
        # Check last 4 candles
        recent_df = df.iloc[-lookback:]
        recent_high = recent_df['high'].max()
        recent_ath_idx = recent_df['high'].idxmax()
        
        has_new_ath = recent_high > previous_ath
        return has_new_ath, (previous_ath, previous_ath_idx), (recent_high, recent_ath_idx)
    
    def create_timeframe_annotations(timeframe, market_cap, volume):
        formatted_volume = ATH_Breakout_Master.format_number(volume)
        return [
            dict(
                text=f"<b>Timeframe:</b> {timeframe}",
                x=0.2,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            ),
            dict(
                text=f"<b>Current MC:</b> {market_cap}",
                x=0.5,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            ),
            dict(
                text=f"<b>Prior Candle Vol:</b> {formatted_volume}",
                x=0.8,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            )
        ]
    
    def has_outliers(df, threshold_multiplier=15.0):
        if df.empty:
            return True
            
        median_price = df['close'].median()
        high_outliers = df['high'] > (median_price * threshold_multiplier)
        
        if not high_outliers.any():
            return False
            
        outlier_indices = df.index[high_outliers]
        for idx in outlier_indices:
            start_idx = max(0, df.index.get_loc(idx) - 3)
            end_idx = min(len(df), df.index.get_loc(idx) + 4)
            window = df.iloc[start_idx:end_idx]
            
            window_median = window['close'].median()
            if df.loc[idx, 'high'] > window_median * threshold_multiplier:
                return True
                
        return False
    
    # Define color scheme with fainter grid
    BACKGROUND_COLOR = '#1e1e1e'
    TEXT_COLOR = '#ffffff'
    GRID_COLOR = 'rgba(51, 51, 51, 0.3)'
    TITLE_COLOR = '#00cc00'
    PREVIOUS_ATH_COLOR = '#FFD700'  # Gold for previous ATH
    NEW_ATH_COLOR = '#FF0000'       # Bright red for new ATH
    
    available_timeframes = ['1m', '5m', '15m', '1H']
    token_data = []
    
    # Process market cap data
    base_path = 'Data/New_Token_Data'
    date_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f.startswith('2024_')]
    most_recent_date = max(date_folders)
    date_folder = os.path.join(base_path, most_recent_date)
    
    summary_folder = os.path.join(date_folder, 'Token_Summary')
    datetime_folders = [f for f in os.listdir(summary_folder) if os.path.isdir(os.path.join(summary_folder, f))]
    most_recent_datetime = max(datetime_folders)
    summary_datetime_folder = os.path.join(summary_folder, most_recent_datetime)
    
    mc_files = [f for f in os.listdir(summary_datetime_folder) if f.startswith('new_tokens_mc') and f.endswith('.csv')]
    latest_mc_file = os.path.join(summary_datetime_folder, mc_files[0])
    market_caps = pd.read_csv(latest_mc_file, index_col=0)['Market Cap']
    
    # Calculate and filter tokens with new ATHs
    for token_address, timeframes in imported_ohlcv_data.items():
        if initial_timeframe not in timeframes:
            continue
            
        df_initial = timeframes[initial_timeframe]
        if df_initial.empty or has_outliers(df_initial):
            continue
            
        has_new_ath, previous_ath_data, recent_ath_data = check_recent_ath(df_initial)
        if has_new_ath:
            token_data.append({
                'address': token_address,
                'previous_ath_data': previous_ath_data,
                'recent_ath_data': recent_ath_data
            })
    
    # Create plots for each token
    for token_data in token_data:
        token_address = token_data['address']
        gmgn_link = f"https://gmgn.ai/sol/token/{token_address}"
        dexscreener_link = f"https://dexscreener.com/solana/{token_address}"
        print(f"\nProcessing: {gmgn_link}\n{dexscreener_link}")
        
        timeframes = imported_ohlcv_data[token_address]
        market_cap = ATH_Breakout_Master.format_number(market_caps.get(token_address, 0))
        
        fig = go.Figure(layout_xaxis_rangeslider_visible=False)
        timeframe_traces = {}
        timeframe_volumes = {}
        
        # Process each timeframe
        for tf in available_timeframes:
            if tf not in timeframes:
                continue
                
            df = timeframes[tf]
            if df.empty:
                continue
                
            traces = []
            timeframe_volumes[tf] = df['volume'].iloc[-1]
            
            # Add candlestick chart
            traces.append(go.Candlestick(
                x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name="Price",
                showlegend=False,
                increasing_line_color='#00ff9d',
                decreasing_line_color='#ff005b',
                visible=(tf == initial_timeframe)
            ))
            
            # Add previous ATH star
            prev_ath, prev_idx = token_data['previous_ath_data']
            traces.append(go.Scatter(
                x=[prev_idx],
                y=[prev_ath],
                mode='markers',
                marker=dict(
                    symbol='star',
                    size=24,
                    color=PREVIOUS_ATH_COLOR,
                    line=dict(
                        color=PREVIOUS_ATH_COLOR,
                        width=2
                    )
                ),
                name='Previous ATH',
                showlegend=True,
                visible=(tf == initial_timeframe)
            ))
            
            # Add new ATH star
            recent_ath, recent_idx = token_data['recent_ath_data']
            traces.append(go.Scatter(
                x=[recent_idx],
                y=[recent_ath],
                mode='markers',
                marker=dict(
                    symbol='star',
                    size=24,
                    color=NEW_ATH_COLOR,
                    line=dict(
                        color=NEW_ATH_COLOR,
                        width=2
                    )
                ),
                name='New ATH',
                showlegend=True,
                visible=(tf == initial_timeframe)
            ))
            
            timeframe_traces[tf] = traces
        
        # Create mobile-friendly timeframe buttons
        buttons = []
        for tf in available_timeframes:
            if tf not in timeframe_traces:
                continue
                
            visible_traces = []
            for other_tf in timeframe_traces:
                visible_traces.extend([tf == other_tf] * len(timeframe_traces[other_tf]))
            
            tf_annotations = create_timeframe_annotations(
                tf,
                market_cap,
                timeframe_volumes[tf]
            )
            
            buttons.append(dict(
                label=tf,
                method="update",
                args=[
                    {"visible": visible_traces},
                    {"annotations": tf_annotations}
                ]
            ))
        
        # Add all traces to the figure
        for traces in timeframe_traces.values():
            fig.add_traces(traces)
        
        # Create initial annotations
        initial_annotations = create_timeframe_annotations(
            initial_timeframe,
            market_cap,
            timeframe_volumes[initial_timeframe]
        )
        
        # Update layout with fixed dimensions
        fig.update_layout(
            height=800,
            width=1200,
            paper_bgcolor=BACKGROUND_COLOR,
            plot_bgcolor=BACKGROUND_COLOR,
            margin=dict(r=50, t=80, l=50, b=50),
            showlegend=True,  # Changed to True to show ATH stars in legend
            hovermode='x unified',
            annotations=initial_annotations,
            updatemenus=[dict(
                type="buttons",
                direction="right",
                x=0.5,
                y=1.05,
                xanchor="center",
                yanchor="bottom",
                showactive=True,
                active=available_timeframes.index(initial_timeframe),
                buttons=buttons,
                bgcolor=BACKGROUND_COLOR,
                bordercolor="#555555",
                borderwidth=2,
                font=dict(
                    size=16,
                    color=TITLE_COLOR
                ),
                pad=dict(r=20, t=10, b=10, l=20)
            )],
            xaxis=dict(
                showticklabels=False,
                showgrid=True,
                gridcolor=GRID_COLOR,
                showline=True,
                linecolor=GRID_COLOR,
                rangeslider=dict(visible=False)
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor=GRID_COLOR,
                showticklabels=False,
                showline=False,
                zeroline=False,
                fixedrange=True
            )
        )
        
        fig.show()

# Call the function
plot_ath_breakouts(imported_ohlcv_data, initial_timeframe='5m')